### CRUD - Create 구현


- CRUD중 Create부분 구현
- HTTP method중 POST에 해당

#### 1. serializer  생성하기
- 기존 Read에서와 동일하게 serializer를 생성해야한다
- POST를 위한 serializer로 따로 생성

In [ ]:
# blog -> serializers.py

class BlogPostPutSerializer(serializers.ModelSerializer): # Post를 위한 serializer
    class Meta:
        model = Blog
        fields = ['id', 'title', 'body']
        read_only_fields = ['id'] # read만 가능

In [ ]:
# blog -> views.py

from .serializers import BlogSerializer, BlogPostPutSerializer # BlogPostPutSerializer 추가

@api_view(['POST']) # 해당함수에서는 POST라는 request method를 사용
def post_one_blog(request):
    serializer = BlogPostPutSerializer(data=request.data) # BlogPostPutSerializer 에 요청이 들어온 데이터를 넘겨 serializer 라는 변수에 담는다.
    if serializer.is_valid(): # 유효성 검사
        serializer.save() # 저장
        return Response(serializer.data, status = status.HTTP_201_CREATED) # 유효할 경우 상태코드 201 반환
    return Response(status=status.HTTP_400_BAD_REQUEST) # 유효하지 않을 경우 상태코드 400 반환

- 201 상태코드:  요청이 성공적으로 처리되었으며, 자원이 생성되었음을 나타내는 성공 상태 응답 코드


- 400 상태코드:  서버가 클라이언트 오류(예: 잘못된 요청 구문, 유효하지 않은 요청 메시지 프레이밍, 또는 변조된 요청 라우팅) 를 감지해 요청을 처리할 수 없거나, 하지 않는다는 응답 코드

#### 2. urls.py 추가하기
- views.py 수정후 urls.py 반드시 수정하기

In [ ]:
# blog -> urls.py

from django.urls import path
from .views import *

app_name = 'blog'

urlpatterns = [
    path('', get_all_blogs),
    path('post', post_one_blog), # 추가
    path('<int:pk>/', get_one_blog),
]

### CRUD - Update 구현


- 별도의 serializer 사용x
- 기존의 Create의 serializer 사용

#### 1. views.py 작성


- api_view 데코레이터의 경우는 'PUT' 사용
- pk를 통해 글이 있는지 확인
- BlogPostPutSerializer(blog, data=request.data) -> 이미 존재하는 블로그 객체를 수정하고 요청이 들어온 데이터를 넘김

In [ ]:
# blog -> views.py

@api_view(['PUT']) # update -> 'PUT'사용
def put_one_blog(request, pk): # pk를 통해 수정하는 글이 있는지 확인
    try: # 오류가 없을 경우 
        blog = Blog.objects.get(pk=pk)
        serializer = BlogPostPutSerializer(blog, data=request.data) # 이미 존재하는 블로그 객체 수정, 요청이 들어온 데이터 넘김
        if serializer.is_valid():
            serializer.save()
            return Response(status=status.HTTP_200_OK) # 200코드 반환
        return Response(status=status.HTTP_400_BAD_REQUEST) # 유효하지 않을 경우 -> 400코드 반환
    except Blog.DoesNotExist: # 블로그가 존재하지 않을 경우
        return Response(status=status.HTTP_404_NOT_FOUND) # 404코드 반환

- 200 상태코드: 서버가 요청을 제대로 처리했다는 코드


- 404 상태코드: 요청한 페이지를 찾을수 없다는 코드

#### 2. urls.py 추가하기

In [ ]:
from django.urls import path
from .views import *


app_name = 'blog'

urlpatterns = [
    path('', get_all_blogs),
    path('post', post_one_blog),
    path('<int:pk>/', get_one_blog),
    path('put/<int:pk>/', put_one_blog), # 추가 / 1번째 글 -> 'pk = 1' 
]

### CRUD - Delete 구현
- 삭제 -> serializer 필요없음

#### 1. views.py 작성하기


- api_view 데코레이터는 'DELETE' 사용
- pk를 통해 블로그 존재여부 확인

In [ ]:
# blog -> views.py

@api_view(['DELETE']) # delete -> 'DELETE' 사용
def delete_one_blog(request, pk):# pk를 통해 존재여부 확인
    try: # 블로그가 있을 경우
        blog = Blog.objects.get(pk=pk)
        blog.delete() # 삭제
        return Response(status=status.HTTP_204_NO_CONTENT) # 204 코드 반환
    except Blog.DoesNotExist: # 블로그가 없을 경우
        return Response(status=status.HTTP_404_NOT_FOUND) # 404 코드 반환

- 204 상태코드: 요청이 성공했으나 클라이언트가 현재 페이지에서 벗어나지 않아도 된다는 것을 나타내는 코드

#### 2. urls.py 추가하기

In [ ]:
from django.urls import path
from .views import *

app_name = 'blog'

urlpatterns = [
    path('', get_all_blogs),
    path('post', post_one_blog),
    path('<int:pk>/', get_one_blog),
    path('put/<int:pk>/', put_one_blog),
    path('delete/<int:pk>/', delete_one_blog), # 추가
]

### 서버와 postman으로 둘다 확인